<a href="https://colab.research.google.com/github/yebyyy/English-Premier-League-2023-2024-Prediction/blob/main/English_Premier_League_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [ ]:
matches = pd.read_csv("matches.csv", index_col=0)
matches

In [5]:
matches.shape

(1608, 27)

#### Checking missing data

In [6]:
38*20*3

2280

In [10]:
matches["team"].value_counts()

Liverpool                   81
Wolverhampton Wanderers     81
Burnley                     81
Nottingham Forest           81
Everton                     81
Brentford                   81
Crystal Palace              81
Fulham                      81
Manchester City             81
Sheffield United            81
Brighton and Hove Albion    81
Newcastle United            81
West Ham United             81
Manchester United           81
Aston Villa                 81
Arsenal                     81
Bournemouth                 78
Tottenham Hotspur           78
Luton Town                  78
Chelsea                     78
Name: team, dtype: int64

In [11]:
matches[matches["team"] == "Chelsea"]

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Home,D,1.0,1.0,Liverpool,...,Match Report,NaN,10.0,4.0,13.2,1.0,0,0,2024,Chelsea
1,2023-08-20,16:30,Premier League,Matchweek 2,Sun,Away,L,1.0,3.0,West Ham,...,Match Report,NaN,16.0,3.0,17.1,0.0,0,1,2024,Chelsea
2,2023-08-25,20:00,Premier League,Matchweek 3,Fri,Home,W,3.0,0.0,Luton Town,...,Match Report,NaN,19.0,8.0,14.2,1.0,0,0,2024,Chelsea
4,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,21.0,2.0,17.7,2.0,0,0,2024,Chelsea
5,2023-09-17,14:00,Premier League,Matchweek 5,Sun,Away,D,0.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,15.0,1.0,0,0,2024,Chelsea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,2024-01-31,20:15,Premier League,Matchweek 22,Wed,Away,L,1.0,4.0,Liverpool,...,Match Report,NaN,4.0,3.0,18.8,0.0,0,0,2022,Chelsea
30,2024-02-04,14:00,Premier League,Matchweek 23,Sun,Home,L,2.0,4.0,Wolves,...,Match Report,NaN,15.0,4.0,14.5,0.0,0,0,2022,Chelsea
32,2024-02-12,20:00,Premier League,Matchweek 24,Mon,Away,W,3.0,1.0,Crystal Palace,...,Match Report,NaN,14.0,5.0,15.3,0.0,0,0,2022,Chelsea
33,2024-02-17,17:30,Premier League,Matchweek 25,Sat,Away,D,1.0,1.0,Manchester City,...,Match Report,NaN,9.0,6.0,12.8,0.0,0,0,2022,Chelsea


#### Data Cleaning

In [12]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk                int64
pkatt             int64
season            int64
team             object
dtype: object

Cast date from string to numeric value

In [13]:
matches["date"] = pd.to_datetime(matches["date"])

In [ ]:
matches

#### Base predictors

In [15]:
# converting "home"/"away" to a numeric column
matches["veneu_code"] = matches["venue"].astype("category").cat.codes

In [17]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [19]:
# remove the colon and the minutes
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [22]:
matches["day_code"] = matches["date"].dt.day_of_week

Prediction target: Whether the team won or not

In [26]:
matches["target"] = (matches["result"] == "W").astype("int")

In [27]:
matches

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,veneu_code,opp_code,hour,day_code,target
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,0.0,0,0,2024,Liverpool,0,6,16,6,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,1.0,0,1,2024,Liverpool,1,2,15,5,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,1.0,0,0,2024,Liverpool,0,14,16,6,1
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,0.0,0,0,2024,Liverpool,1,1,14,6,1
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,0.0,0,0,2024,Liverpool,0,19,12,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,2024-02-03,17:30,Premier League,Matchweek 23,Sat,Home,L,0.0,5.0,Aston Villa,...,0.0,0,0,2022,Sheffield United,1,1,17,5,0
26,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Away,W,3.0,1.0,Luton Town,...,0.0,1,1,2022,Sheffield United,0,11,15,5,1
27,2024-02-18,14:00,Premier League,Matchweek 25,Sun,Home,L,0.0,5.0,Brighton,...,0.0,0,0,2022,Sheffield United,1,4,14,6,0
28,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Away,L,0.0,1.0,Wolves,...,0.0,0,0,2022,Sheffield United,0,19,13,6,0


### Initial Machine Learning Model

In [28]:
from sklearn.ensemble import RandomForestClassifier
# random forest classifier can pick up non-linear relationship in the data: such as opp_code

In [29]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

Time series data: test set data comes after train set data

In [45]:
train = matches[matches["date"] < "2024-01-01"]
train

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,veneu_code,opp_code,hour,day_code,target
0,2023-08-13,16:30,Premier League,Matchweek 1,Sun,Away,D,1.0,1.0,Chelsea,...,0.0,0,0,2024,Liverpool,0,6,16,6,0
1,2023-08-19,15:00,Premier League,Matchweek 2,Sat,Home,W,3.0,1.0,Bournemouth,...,1.0,0,1,2024,Liverpool,1,2,15,5,1
2,2023-08-27,16:30,Premier League,Matchweek 3,Sun,Away,W,2.0,1.0,Newcastle Utd,...,1.0,0,0,2024,Liverpool,0,14,16,6,1
3,2023-09-03,14:00,Premier League,Matchweek 4,Sun,Home,W,3.0,0.0,Aston Villa,...,0.0,0,0,2024,Liverpool,1,1,14,6,1
4,2023-09-16,12:30,Premier League,Matchweek 5,Sat,Away,W,3.0,1.0,Wolves,...,0.0,0,0,2024,Liverpool,0,19,12,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2023-12-09,15:00,Premier League,Matchweek 16,Sat,Home,W,1.0,0.0,Brentford,...,0.0,0,0,2022,Sheffield United,1,3,15,5,1
17,2023-12-16,15:00,Premier League,Matchweek 17,Sat,Away,L,0.0,2.0,Chelsea,...,1.0,0,0,2022,Sheffield United,0,6,15,5,0
18,2023-12-22,20:00,Premier League,Matchweek 18,Fri,Away,D,1.0,1.0,Aston Villa,...,0.0,0,0,2022,Sheffield United,0,1,20,4,0
19,2023-12-26,15:00,Premier League,Matchweek 19,Tue,Home,L,2.0,3.0,Luton Town,...,1.0,0,0,2022,Sheffield United,1,11,15,1,0


In [44]:
test = matches[matches["date"] >= "2024-01-01"]
test

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,fk,pk,pkatt,season,team,veneu_code,opp_code,hour,day_code,target
28,2024-01-01,20:00,Premier League,Matchweek 20,Mon,Home,W,4.0,2.0,Newcastle Utd,...,0.0,1,2,2024,Liverpool,1,14,20,0,1
31,2024-01-21,16:30,Premier League,Matchweek 21,Sun,Away,W,4.0,0.0,Bournemouth,...,0.0,0,0,2024,Liverpool,0,2,16,6,1
34,2024-01-31,20:15,Premier League,Matchweek 22,Wed,Home,W,4.0,1.0,Chelsea,...,2.0,0,1,2024,Liverpool,1,6,20,2,1
35,2024-02-04,16:30,Premier League,Matchweek 23,Sun,Away,L,1.0,3.0,Arsenal,...,0.0,0,0,2024,Liverpool,0,0,16,6,0
36,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Home,W,3.0,1.0,Burnley,...,0.0,0,0,2024,Liverpool,1,5,15,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,2024-02-03,17:30,Premier League,Matchweek 23,Sat,Home,L,0.0,5.0,Aston Villa,...,0.0,0,0,2022,Sheffield United,1,1,17,5,0
26,2024-02-10,15:00,Premier League,Matchweek 24,Sat,Away,W,3.0,1.0,Luton Town,...,0.0,1,1,2022,Sheffield United,0,11,15,5,1
27,2024-02-18,14:00,Premier League,Matchweek 25,Sun,Home,L,0.0,5.0,Brighton,...,0.0,0,0,2022,Sheffield United,1,4,14,6,0
28,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Away,L,0.0,1.0,Wolves,...,0.0,0,0,2022,Sheffield United,0,19,13,6,0


In [46]:
predictors = ["veneu_code", "opp_code", "hour", "day_code"]

In [47]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [48]:
preds = rf.predict(test[predictors])

In [50]:
from sklearn.metrics import accuracy_score

In [51]:
acc = accuracy_score(test["target"], preds)

In [52]:
acc

0.5416666666666666

In [53]:
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

In [54]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,159,108
1,90,75


In [55]:
from sklearn.metrics import precision_score

In [56]:
precision = precision_score(test["target"], preds)
precision

0.4098360655737705

#### Improving accuracy through rolling averages
Use the past week's performances to predict the performance on the next week

In [58]:
# create a dataframe for every team
grouped_matches = matches.groupby("team")


In [59]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  #closed=left: take out the current week data
  rolling_stats = group[cols].rolling(3, closed="left").mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group


In [60]:
cols = ["ga", "gf", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f'{c}_rolling' for c in cols]
new_cols

['ga_rolling',
 'gf_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [61]:
group = grouped_matches.get_group("Manchester City")

In [72]:
matches_rollin = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [67]:
matches_rollin

date   time            comp         round  \
team                                                                         
Arsenal                 2  2023-08-21  20:00  Premier League   Matchweek 2   
                        2  2023-08-21  20:00  Premier League   Matchweek 2   
                        2  2023-08-21  20:00  Premier League   Matchweek 2   
                        3  2023-08-26  15:00  Premier League   Matchweek 3   
                        3  2023-08-26  15:00  Premier League   Matchweek 3   
...                               ...    ...             ...           ...   
Wolverhampton Wanderers 30 2024-02-25  13:30  Premier League  Matchweek 26   
                        30 2024-02-25  13:30  Premier League  Matchweek 26   
                        32 2024-03-02  15:00  Premier League  Matchweek 27   
                        32 2024-03-02  15:00  Premier League  Matchweek 27   
                        32 2024-03-02  15:00  Premier League  Matchweek 27   

                            day venue result   gf   ga        opponent  ...  \
team                                                                    ...   
Arsenal                 2   Mon  Away      W  1.0  0.0  Crystal Palace  ...   
                        2   Mon  Away      W  1.0  0.0  Crystal Palace  ...   
                        2   Mon  Away      W  1.0  0.0  Crystal Palace  ...   
                        3   Sat  Home      D  2.0  2.0          Fulham  ...   
                        3   Sat  Home      D  2.0  2.0          Fulham  ...   
...                         ...   ...    ...  ...  ...             ...  ...   
Wolverhampton Wanderers 30  Sun  Home      W  1.0  0.0   Sheffield Utd  ...   
                        30  Sun  Home      W  1.0  0.0   Sheffield Utd  ...   
                        32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   
                        32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   
                        32  Sat  Away      L  0.0  3.0   Newcastle Utd  ...   

                            day_code  target  ga_rolling  gf_rolling  \
team                                                                   
Arsenal                 2          0       1    1.000000    2.000000   
                        2          0       1    0.666667    1.666667   
                        2          0       1    0.333333    1.333333   
                        3          5       0    0.000000    1.000000   
                        3          5       0    0.666667    1.333333   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 30         6       1    0.666667    1.666667   
                        30         6       1    0.333333    1.333333   
                        32         5       0    0.000000    1.000000   
                        32         5       0    1.000000    0.666667   
                        32         5       0    2.000000    0.333333   

                           sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                        
Arsenal                 2   15.000000    7.000000    19.100000        0.0   
                        2   14.333333    5.333333    18.200000        0.0   
                        2   13.666667    3.666667    17.300000        0.0   
                        3   13.000000    2.000000    16.400000        0.0   
                        3   14.666667    4.333333    15.533333        0.0   
...                               ...         ...          ...        ...   
Wolverhampton Wanderers 30  12.333333    5.333333    15.466667        0.0   
                        30  12.666667    3.666667    15.033333        0.0   
                        32  13.000000    2.000000    14.600000        0.0   
                        32  12.666667    2.333333    15.033333        0.0   
                        32  12.333333    2.666667    15.466667        0.0   

                            pk_rolling  pkatt_rolli

In [74]:
matches_rollin = matches_rollin.droplevel("team")
matches_rollin

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,ga_rolling,gf_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,1.000000,2.000000,15.000000,7.000000,19.100000,0.0,0.000000,0.000000
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,0.666667,1.666667,14.333333,5.333333,18.200000,0.0,0.333333,0.333333
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,0.333333,1.333333,13.666667,3.666667,17.300000,0.0,0.666667,0.666667
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,0.000000,1.000000,13.000000,2.000000,16.400000,0.0,1.000000,1.000000
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,0.666667,1.333333,14.666667,4.333333,15.533333,0.0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,0.666667,1.666667,12.333333,5.333333,15.466667,0.0,0.000000,0.000000
30,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,0.333333,1.333333,12.666667,3.666667,15.033333,0.0,0.000000,0.000000
32,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,0,0.000000,1.000000,13.000000,2.000000,14.600000,0.0,0.000000,0.000000
32,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,0,1.000000,0.666667,12.666667,2.333333,15.033333,0.0,0.000000,0.000000


In [76]:
# reassign indices
matches_rollin.index = range(matches_rollin.shape[0])
matches_rollin

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,ga_rolling,gf_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,1.000000,2.000000,15.000000,7.000000,19.100000,0.0,0.000000,0.000000
1,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,0.666667,1.666667,14.333333,5.333333,18.200000,0.0,0.333333,0.333333
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1.0,0.0,Crystal Palace,...,0,1,0.333333,1.333333,13.666667,3.666667,17.300000,0.0,0.666667,0.666667
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,0.000000,1.000000,13.000000,2.000000,16.400000,0.0,1.000000,1.000000
4,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2.0,2.0,Fulham,...,5,0,0.666667,1.333333,14.666667,4.333333,15.533333,0.0,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,0.666667,1.666667,12.333333,5.333333,15.466667,0.0,0.000000,0.000000
1544,2024-02-25,13:30,Premier League,Matchweek 26,Sun,Home,W,1.0,0.0,Sheffield Utd,...,6,1,0.333333,1.333333,12.666667,3.666667,15.033333,0.0,0.000000,0.000000
1545,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,0,0.000000,1.000000,13.000000,2.000000,14.600000,0.0,0.000000,0.000000
1546,2024-03-02,15:00,Premier League,Matchweek 27,Sat,Away,L,0.0,3.0,Newcastle Utd,...,5,0,1.000000,0.666667,12.666667,2.333333,15.033333,0.0,0.000000,0.000000


### Retrain by new predictors

In [79]:
def make_predictions(data, predictors):
  train = matches_rollin[matches_rollin["date"] < "2024-01-01"]
  test = matches_rollin[matches_rollin["date"] >= "2024-01-01"]
  rf.fit(train[predictors], train["target"])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))
  precision = precision_score(test["target"], preds)
  return combined, precision

In [80]:
combined, precision = make_predictions(matches_rollin, predictors + new_cols)

In [81]:
precision

0.5935483870967742

In [84]:
combined = combined.merge(matches_rollin[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [86]:
combined

,actual,prediction,date,team,opponent,result
57,1,0,2024-01-20,Arsenal,Crystal Palace,W
58,1,1,2024-01-20,Arsenal,Crystal Palace,W
59,1,1,2024-01-20,Arsenal,Crystal Palace,W
60,1,0,2024-01-30,Arsenal,Nott'ham Forest,W
61,1,1,2024-01-30,Arsenal,Nott'ham Forest,W
...,...,...,...,...,...,...
1543,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W
1544,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W
1545,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L
1546,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L


### Combining home and away results

In [87]:
class MissingDict(dict):
  __missing__ = lambda self, key: key

mapping_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcastle Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**mapping_values)

In [89]:
mapping["Manchester United"]

'Manchester Utd'

In [90]:
combined["new_team"] = combined["team"].map(mapping)

In [91]:
combined

,actual,prediction,date,team,opponent,result,new_team
57,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
58,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
59,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal
60,1,0,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
61,1,1,2024-01-30,Arsenal,Nott'ham Forest,W,Arsenal
...,...,...,...,...,...,...,...
1543,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves
1544,1,0,2024-02-25,Wolverhampton Wanderers,Sheffield Utd,W,Wolves
1545,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves
1546,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves


In [93]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
1,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
2,1,0,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
3,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
4,1,1,2024-01-20,Arsenal,Crystal Palace,W,Arsenal,0,0,Crystal Palace,Arsenal,L,Crystal Palace
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd
1166,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,1,Newcastle United,Wolves,W,Newcastle Utd
1167,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd
1168,0,0,2024-03-02,Wolverhampton Wanderers,Newcastle Utd,L,Wolves,1,0,Newcastle United,Wolves,W,Newcastle Utd


In [99]:
merged[(merged["prediction_x"] == 1) & (merged ["prediction_y"] == 0)]["actual_x"].value_counts()

1    215
0    110
Name: actual_x, dtype: int64

In [102]:
215/(215+110)

0.6615384615384615